In [59]:
#################  labelme to tfrecored ,2020-6-24 10:54

In [72]:
import os,cv2
import shutil

In [61]:
data_root = "E:/HongsenQu/2_project/2_projects/images_output/output/new_output"

images_dir = os.path.join(data_root,'images')
jsons_dir = os.path.join(data_root,'jsons')
mask_dir = os.path.join(data_root,'cv2_mask')

In [62]:
imageList = [file_name  for file_name in os.listdir(images_dir) if file_name.endswith('.jpg')]
jsonList  = [file_name  for file_name in os.listdir(jsons_dir) if file_name.endswith('.json')]
maskList = [file_name  for file_name in os.listdir(mask_dir) if file_name.endswith('_mask.png')]

In [64]:
## ###################  check mask,json,image:  检查文件匹配情况。
n = 0
for file_name in maskList:
    file_id = str(file_name).replace("_mask.png",'')
    
    if file_id+'.jpg' in imageList and file_id+".json" in jsonList:
        n+=1
    else:
        print("check: %s, %s"%(file_id+'.jpg',file_id+".json"))
print("num_mask: %d , match num: %d" %(len(maskList),n))
    

num_mask: 405 , match num: 405


In [110]:
##### labelme mask to binary mask，将labelme mask 转化为二值图，分为测试和训练

bin_mask_dir = os.path.join(data_root,'cv2_bin_mask')
train_mask = os.path.join(data_root,'split_dat','train_mask')
val_mask = os.path.join(data_root,'split_dat','val_mask')

if not os.path.exists(train_mask):os.makedirs(train_mask)
if not os.path.exists(val_mask):os.makedirs(val_mask)


def get_bin_mask(mask_in_dir,out_dir):
    num = 0
    for file_name in os.listdir(mask_in_dir):
        if file_name.endswith("_mask.png"):
            mask_in = os.path.join(mask_in_dir,file_name)
            img = cv2.imread(mask_in)
            gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
            ret,th = cv2.threshold(gray,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
            save_mask = os.path.join(out_dir,file_name)
            cv2.imwrite(save_mask,th)
            if os.path.exists(save_mask):
                print("Done: %s"%file_name)
                num +=1
    print("input: %d and output : %d" %(len(maskList),num))        


def copy_mask_image(mask_dir,save_dir,jsonList):
    num =0
    for file_name in jsonList:
        mask_name = os.path.join(mask_dir,file_name.replace(".json","_mask.png"))
        if os.path.exists(mask_name):
            # save_mask = os.path.join(save_dir,os.path.basename(mask_name))
            save_mask = os.path.join(save_dir,file_name.replace(".json",".png"))
            shutil.copy(mask_name,save_mask)
            if os.path.exists(save_mask):
                num+=1
                # print("done: %s"% file_name)
        else:
            print("Not Find: %s"%mask_name)
    print("mask num: %d ,copy num: %d"% (len(jsonList),num))
    
# get_bin_mask(mask_dir,bin_mask_dir)
train_jsonList = [file_name for file_name in os.listdir(os.path.join(data_root,'split_dat','train_jsons')) if file_name.endswith('.json')]
val_jsonList = [file_name for file_name in os.listdir(os.path.join(data_root,'split_dat','val_jsons')) if file_name.endswith('.json')]

copy_mask_image(mask_dir,'E:\\HongsenQu\\2_project\\2_projects\\images_output\\output\\new_output\\split_dat\\11',val_jsonList)



mask num: 43 ,copy num: 43


In [27]:
#############################  labelme json 转为 xml格式,然后再转化为tf.record 【需要调试，mask rcnn 预测只有box,无mask】
#### 参考：https://blog.csdn.net/weixin_43211480/article/details/106481851


from glob import glob
import json
import codecs
import numpy as np

import hashlib
import io
import logging
import contextlib2
import PIL.Image
import tensorflow as tf
from lxml import etree

from object_detection.dataset_tools import tf_record_creation_util
from object_detection.utils import dataset_util
from object_detection.utils import label_map_util



def labelme_annotation_to_coco_xmls(json_dir,image_dir,out_root):
    
    """
    :param json_dir: labelme 生成json文件所在目录,"E:/HongsenQu/2_project/2_projects/images_output/output/new_output/split_dat/train_jsons/"
    :param image_dir: 原始标注图像目录,'E:/HongsenQu/2_project/2_projects/images_output/output/new_output/split_dat/train_images/'
    :param mask_dir: labelme输出mask路径
    :param out_root: 输出父目录
    :return: 
    """
    
    ### 输出结果文件夹
    save_anno_dir = os.path.join(out_root,"Annotations")
    save_img_dir = os.path.join(out_root,"JPEGImages")
    save_main_dir = os.path.join(out_root,"ImageSets/Main/")
    
    if not os.path.exists(save_anno_dir):os.makedirs(save_anno_dir)
    if not os.path.exists(save_img_dir):os.makedirs(save_img_dir)
    if not os.path.exists(save_main_dir):os.makedirs(save_main_dir)
    
    
    #####获取待处理文件
    jsonFiles = glob(json_dir + "*.json")
    jsonFiles = [i.replace("\\","/").split("/")[-1].split(".json")[0] for i in jsonFiles]
    # print(jsonFiles)
    ####读取标注信息并写入 xml
    for file_name in jsonFiles:
        json_filename = json_dir + file_name + ".json"
        image_filename = image_dir + file_name + ".jpg"
        print(file_name)
        json_file = json.load(open(json_filename, "r", encoding="utf-8"))
        saveXML_filename = os.path.join(save_anno_dir, file_name + ".xml")
        height, width, channels = cv2.imread(image_filename).shape
        with codecs.open(saveXML_filename, "w", "utf-8") as xml:
            xml.write('<annotation>\n')
            xml.write('\t<folder>' + 'labelme_data' + '</folder>\n')
            xml.write('\t<filename>' + str(os.path.basename(image_filename)) + '</filename>\n')
            xml.write('\t<source>\n')
            xml.write('\t\t<database>labelme Data</database>\n')
            xml.write('\t\t<annotation>labelme</annotation>\n')
            xml.write('\t\t<image>NULL</image>\n')
            xml.write('\t\t<flickrid>NULL</flickrid>\n')
            xml.write('\t</source>\n')
            xml.write('\t<owner>\n')
            xml.write('\t\t<flickrid>NULL</flickrid>\n')
            xml.write('\t\t<name>NULL</name>\n')
            xml.write('\t</owner>\n')
            xml.write('\t<size>\n')
            xml.write('\t\t<width>' + str(width) + '</width>\n')
            xml.write('\t\t<height>' + str(height) + '</height>\n')
            xml.write('\t\t<depth>' + str(channels) + '</depth>\n')
            xml.write('\t</size>\n')
            xml.write('\t\t<segmented>0</segmented>\n')
            for multi in json_file["shapes"]:
                points = np.array(multi["points"])
                labelName=multi["label"]
                xmin = min(points[:, 0])
                xmax = max(points[:, 0])
                ymin = min(points[:, 1])
                ymax = max(points[:, 1])
                label = multi["label"]
                if xmax <= xmin:
                    pass
                elif ymax <= ymin:
                    pass
                else:
                    xml.write('\t<object>\n')
                    xml.write('\t\t<name>' + labelName+ '</name>\n')
                    xml.write('\t\t<pose>Unspecified</pose>\n')
                    xml.write('\t\t<truncated>1</truncated>\n')
                    xml.write('\t\t<difficult>0</difficult>\n')
                    xml.write('\t\t<bndbox>\n')
                    xml.write('\t\t\t<xmin>' + str(int(xmin)) + '</xmin>\n')
                    xml.write('\t\t\t<ymin>' + str(int(ymin)) + '</ymin>\n')
                    xml.write('\t\t\t<xmax>' + str(int(xmax)) + '</xmax>\n')
                    xml.write('\t\t\t<ymax>' + str(int(ymax)) + '</ymax>\n')
                    xml.write('\t\t</bndbox>\n')
                    xml.write('\t</object>\n')
                    print(json_filename, xmin, ymin, xmax, ymax, label)
            xml.write('</annotation>')
    
    print("copy image to %s ..." % str(save_img_dir))      
    for file_name in jsonFiles:
        image = os.path.join(images_dir, file_name + ".jpg")
        shutil.copy(image,os.path.join(save_img_dir,os.path.basename(image)))
        
####### xml 转化为tf.record 使用 E:\HongsenQu\0_code\from_labelme_create_mask_rcnn_tf_record.py
#### 使用方法： 

def image_to_tf_data(img_path, mask_path,xml_path,label_map_dict,filename):

    with tf.gfile.GFile(img_path, 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image  = PIL.Image.open(encoded_jpg_io)
    width  = np.asarray(image).shape[1]
    height = np.asarray(image).shape[0]
    if image.format != 'JPEG':
        raise ValueError('Image format not JPEG')
    key = hashlib.sha256(encoded_jpg).hexdigest()

    with tf.gfile.GFile(mask_path, 'rb') as fid:
        encoded_mask_png = fid.read()
    encoded_png_io = io.BytesIO(encoded_mask_png)
    mask = PIL.Image.open(encoded_png_io)
    mask_np = np.asarray(mask.convert('L'))
    if mask.format != 'PNG':
        raise ValueError('Mask format not PNG')
  
    data = []
    classes = []
    classes_text = []
    xmins = []
    ymins = []
    xmaxs = []
    ymaxs = []
    encoded_mask_png_list = []
  
  
    with tf.gfile.GFile(xml_path, 'r') as fid:
        xml_str = fid.read()
    xml = etree.fromstring(xml_str)      
    data = dataset_util.recursive_parse_xml_to_dict(xml)['annotation']
     
    if 'object' in data:
        #print("o")
        for obj in data['object']:
            class_name = obj['name']
            #print(label_map_dict)
            #print(class_name)
            #print(label_map_dict[class_name])
            #pixel_val = int(label_map_dict[class_name][1])
            pixel_val = int(label_map_dict[class_name][0]) ## qunosen,2020-6-24
            #print(label_map_dict[class_name][1])
            xmin = float(obj['bndbox']['xmin'])
            xmax = float(obj['bndbox']['xmax'])
            ymin = float(obj['bndbox']['ymin'])
            ymax = float(obj['bndbox']['ymax'])
            print(filename, 'bounding box for', class_name,  xmin, xmax, ymin, ymax)
            xmins.append(xmin / width)
            ymins.append(ymin / height)
            xmaxs.append(xmax / width)
            ymaxs.append(ymax / height)
            
            classes_text.append(class_name.encode('utf8'))
            classes.append(label_map_dict[class_name][0])
            
            mask_np_black = mask_np*0
            mask_np_black[int(ymin):int(ymax), int(xmin):int(xmax)] = mask_np[int(ymin):int(ymax), int(xmin):int(xmax)]
            mask_remapped = (mask_np_black == pixel_val).astype(np.uint8)
            img = PIL.Image.fromarray(mask_remapped)
            output = io.BytesIO()
            img.save(output, format='PNG')
            encoded_mask_png_list.append(output.getvalue())
 
    # print(classes)
    feature_dict = {
      'image/height':             dataset_util.int64_feature(height),
      'image/width':              dataset_util.int64_feature(width),
      'image/filename':           dataset_util.bytes_feature(filename.encode('utf8')),
      'image/source_id':          dataset_util.bytes_feature(filename.encode('utf8')),
      'image/key/sha256':         dataset_util.bytes_feature(key.encode('utf8')),
      'image/encoded':            dataset_util.bytes_feature(encoded_jpg),
      'image/format':             dataset_util.bytes_feature('jpeg'.encode('utf8')),
      'image/object/bbox/xmin':   dataset_util.float_list_feature(xmins),
      'image/object/bbox/xmax':   dataset_util.float_list_feature(xmaxs),
      'image/object/bbox/ymin':   dataset_util.float_list_feature(ymins),
      'image/object/bbox/ymax':   dataset_util.float_list_feature(ymaxs),
      'image/object/class/text':  dataset_util.bytes_list_feature(classes_text),
      'image/object/class/label': dataset_util.int64_list_feature(classes),
      'image/object/mask':        dataset_util.bytes_list_feature(encoded_mask_png_list)}
    tf_data = tf.train.Example(features=tf.train.Features(feature=feature_dict))
    return tf_data

def labelme_xml_to_tfrecord(image_dir_path,annotations_dir_path,tfrecord_dir_path,
                            label_map_dict,images_filename):
    masks_dir_path = os.path.join(annotations_dir_path, 'masks')
    xmls_dir_path = os.path.join(annotations_dir_path, 'xmls')
    num_shards = 1
    with contextlib2.ExitStack() as tf_record_close_stack:
        
        tfrecord_path  = os.path.join(tfrecord_dir_path,'data.tfrecord')
        output_tfrecords = tf_record_creation_util.open_sharded_output_tfrecords(
        tf_record_close_stack, tfrecord_path,num_shards)
        
        for idx, filename in enumerate(images_filename):
            image_path = os.path.join(image_dir_path, filename + '.jpg')
            mask_path = os.path.join(masks_dir_path, filename + '.png')
            xml_path = os.path.join(xmls_dir_path, filename + '.xml')
            
            tf_example = image_to_tf_data(image_path,mask_path,xml_path,label_map_dict,filename)
            
            if tf_example:
                shard_idx = idx % num_shards
                output_tfrecords[shard_idx].write(tf_example.SerializeToString())

def run_to_tfrecord(label_map_path,images_dir_path,annotations_dir_path,tfrecord_dir_path):
    label_map_dict_temp = label_map_util.get_label_map_dict(label_map_path)
    print("label_map_dict_temp: ",label_map_dict_temp)
    label_map_dict = label_map_dict_temp.copy()
    for key in label_map_dict_temp.keys():
        label_map_dict[key] = [label_map_dict_temp[key], key]
        label_map_dict[key] = label_map_dict.pop(key)
        
    images_filename = os.listdir(images_dir_path) 
    for filename in images_filename:
        if filename[-3:] !='jpg':
            del images_filename[images_filename.index(filename)]
    for filename in images_filename:  
        images_filename[images_filename.index(filename)] = filename[0:-4]
    # print(images_filename)
    
    labelme_xml_to_tfrecord(images_dir_path,annotations_dir_path,
                            tfrecord_dir_path,label_map_dict,
                            images_filename)

    
from PIL import Image
import matplotlib.pyplot as plt
def show_tfrecord_images(tfrecord_file):
    filename = tf.train.string_input_producer([tfrecord_file])
    reader = tf.TFRecordReader()
    _,serializer = reader.read(filename)
    print(serializer)
    feature = tf.parse_single_example(serializer,features={
      # 'image/height':           tf.FixedLenFeature([],tf.int64),
      # 'image/width':              tf.FixedLenFeature([],tf.int64),
      'image/filename':           tf.FixedLenFeature([],tf.string),
      # 'image/source_id':          tf.FixedLenFeature([],tf.string),
      # 'image/key/sha256':         tf.FixedLenFeature([],tf.string),
      # 'image/encoded':            tf.FixedLenFeature([],tf.string),
      # 'image/format':             tf.FixedLenFeature([],tf.string),
      # 'image/object/bbox/xmin':   tf.FixedLenFeature([],tf.int64),
      # 'image/object/bbox/xmax':   tf.FixedLenFeature([],tf.int64),
      # 'image/object/bbox/ymin':   tf.FixedLenFeature([],tf.int64),
      # 'image/object/bbox/ymax':   tf.FixedLenFeature([],tf.int64),
      # 'image/object/class/text':  tf.FixedLenFeature([],tf.string),
      # 'image/object/class/label': tf.FixedLenFeature([],tf.string),
      # 'image/object/mask':        tf.FixedLenFeature([],tf.string)
    })
    
    image = tf.decode_raw(feature['image/filename'], tf.uint8)
    # image = tf.cast(image, tf.float32) * (1. / 255) 
    print(image)
    # with tf.Session() as sess:
    #     coord=tf.train.Coordinator()
    #     threads= tf.train.start_queue_runners(coord=coord)
    #     for i in range(5):
    #         example= sess.run([image])
    #         img_show=Image.fromarray(example)
    #         print(img_show)
    #         plt.imshow(img_show)
    #         plt.show()
    #     coord.request_stop()
    #     coord.join(threads)

# show_tfrecord_images('E:/HongsenQu/2_project/2_projects/images_output/output/new_output/od_data/data.tfrecord-00000-of-00001')  

In [28]:
#### 转化产生xml文件
json_dir= 'E:/HongsenQu/2_project/2_projects/images_output/output/new_output/split_dat/val_jsons/'
image_dir = 'E:/HongsenQu/2_project/2_projects/images_output/output/new_output/split_dat/val_images/'
out_root = 'E:/HongsenQu/2_project/2_projects/images_output/output/new_output/od_data/'
# labelme_annotation_to_coco_xmls(json_dir,image_dir,out_root)

label_map_path = "E:\\HongsenQu\\2_project\\2_projects\\images_output\\output\\new_output\\od_data\\label.pbtxt"
test_anno_dir = 'E:\\HongsenQu\\2_project\\2_projects\\images_output\\output\\new_output\\od_data\\val\\Annotations'
# run_to_tfrecord(label_map_path,image_dir,test_anno_dir,out_root)


label_map_dict_temp:  {'pishafa': 1, 'diaodeng': 2}
019193f60d7ffa0f_9-1726 bounding box for diaodeng 135.0 190.0 65.0 317.0
019193f60d7ffa0f_9-1726 bounding box for diaodeng 110.0 156.0 124.0 336.0
019193f60d7ffa0f_9-1726 bounding box for diaodeng 88.0 130.0 167.0 353.0
[2, 2, 2]
0201818d01b4c227_9-7972 bounding box for pishafa 0.0 266.0 451.0 714.0
0201818d01b4c227_9-7972 bounding box for pishafa 709.0 989.0 375.0 656.0
[1, 1]
0321c83401b8e68b_9-7885 bounding box for pishafa 511.0 972.0 352.0 642.0
0321c83401b8e68b_9-7885 bounding box for pishafa 600.0 736.0 288.0 379.0
0321c83401b8e68b_9-7885 bounding box for pishafa 758.0 989.0 295.0 402.0
[1, 1, 1]
0391494c08890b86_9-2857 bounding box for pishafa 167.0 299.0 326.0 428.0
0391494c08890b86_9-2857 bounding box for pishafa 356.0 423.0 338.0 404.0
[1, 1]
0471163d0f678f0c_9-1082 bounding box for diaodeng 0.0 94.0 20.0 226.0
0471163d0f678f0c_9-1082 bounding box for diaodeng 214.0 290.0 0.0 153.0
0471163d0f678f0c_9-1082 bounding box for di

E:\HongsenQu\0_code\TensorFlowObjectDetection\models-1.13.0\research\object_detection\utils\dataset_util.py:75: FutureWarning: The behavior of this method will change in future versions. Use specific 'len(elem)' or 'elem is not None' test instead.
  if not xml:


In [34]:
####### 直接转化labelme json为 tf.record 文件格式。参考：https://blog.csdn.net/jerson_al/article/details/83989944
### https://blog.csdn.net/ah_107/article/details/104800822, [可以运行]

import cv2
import glob
import hashlib
import io
import numpy as np
import os
import PIL.Image
import tensorflow as tf
from object_detection.utils import label_map_util
import json

def int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))
 
def int64_list_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))
 
def bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))
 
def bytes_list_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=value))
 
def float_list_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))
 
 
def label_id_map(path,num_classes):
    label_map = label_map_util.load_labelmap(path)
    # use_display_name,你的pbtxt文件中如果有可能有name和display_name,当为true使用display_name,false时使用name
    categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=num_classes,
                                                                use_display_name=True)
    new_dict ={d['name']:d['id'] for d in categories}
    return new_dict


def _get_annotation_dict(images_dir, annotation_json_path):  
    """Get boundingboxes and masks.
    
    Args:
        images_dir: Path to images directory.
        annotation_json_path: Path to annotated json file corresponding to
            the image. The json file annotated by labelme with keys:
                ['lineColor', 'imageData', 'fillColor', 'imagePath', 'shapes',
                 'flags'].
            
    Returns:
        annotation_dict: A dictionary containing the following keys:
            ['height', 'width', 'filename', 'sha256_key', 'encoded_jpg',
             'format', 'xmins', 'xmaxs', 'ymins', 'ymaxs', 'masks',
             'class_names'].
#            
#    Raises:
#        ValueError: If images_dir or annotation_json_path is not exist.
    """
#    if not os.path.exists(images_dir):
#        raise ValueError('`images_dir` is not exist.')
#    
#    if not os.path.exists(annotation_json_path):
#        raise ValueError('`annotation_json_path` is not exist.')
    
    if (not os.path.exists(images_dir) or
        not os.path.exists(annotation_json_path)):
        return None
    
    with open(annotation_json_path, 'r') as f:
        json_text = json.load(f)
    shapes = json_text.get('shapes', None)
    if shapes is None:
        return None
    image_relative_path = json_text.get('imagePath', None)
    if image_relative_path is None:
        return None
    image_name = image_relative_path.split('/')[-1]
    image_path = os.path.join(images_dir, image_name)
    if not os.path.exists(image_path):
        return None
    # change_jpg = cv2.imread(image_path)
    #
    # cv2.imshow('change_jpg',change_jpg)
    # cv2.waitKey(0)
    # image_format = image_name.split('.')[-1].replace('jpg', 'jpeg')
    with tf.gfile.GFile(image_path, 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image_ = PIL.Image.open(encoded_jpg_io)
    if image_.format != 'JPEG':
        raise ValueError('Image format not JPEG  ')
    image = cv2.imread(image_path)
    height = image.shape[0]
    width = image.shape[1]
    key = hashlib.sha256(encoded_jpg).hexdigest()
    
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    masks = []
    class_names = []
    hole_polygons = []
    # mask = np.zeros(image.shape[:2])
    for mark in shapes:
        class_name = mark.get('label')#### 类别名称
        class_names.append(class_name)
        # class_names.append('ltrcat_dog')
        polygon = mark.get('points')
        # print('polygon == ',polygon)
        polygon = np.array(polygon,dtype=np.int32)
        if class_name == 'hole':
            hole_polygons.append(polygon)
 
        else:
            mask = np.zeros(image.shape[:2], dtype='uint8')
            # mask = np.zeros(image.shape[:2])
            # print('polygon======',[polygon])
            cv2.fillPoly(mask, [polygon], 1)
            masks.append(mask)
            # plt.imshow(mask)
            # plt.show()
            # Boundingbox
            x = polygon[:, 0]
            y = polygon[:, 1]
            xmin = np.min(x)
            xmax = np.max(x)
            ymin = np.min(y)
            ymax = np.max(y)
            xmins.append(float(xmin) / width)
            xmaxs.append(float(xmax) / width)
            ymins.append(float(ymin) / height)
            ymaxs.append(float(ymax) / height)
 
    # Remove holes in mask
 
    for mask in masks:
        cv2.fillPoly(mask, hole_polygons, 0)
    #     # cv2.imshow('mask',mask*255)
    #     # cv2.waitKey(0)
    # # print('masks-----list == ',masks)
    annotation_dict = {'height': height,
                       'width': width,
                       'filename': image_name,
                       'sha256_key': key,
                       'encoded_jpg': encoded_jpg,
                       'format': image_path,
                       'xmins': xmins,
                       'xmaxs': xmaxs,
                       'ymins': ymins,
                       'ymaxs': ymaxs,
                       'masks': masks,
                       'class_names': class_names
                       }
    
    return annotation_dict



def create_tf_example(annotation_dict, label_map_dict=None):
    if annotation_dict is None:
        return None
    if label_map_dict is None:
        raise ValueError('`label_map_dict` is None')
        
    height = annotation_dict.get('height', None)
    width = annotation_dict.get('width', None)
    filename = annotation_dict.get('filename', None)
    sha256_key = annotation_dict.get('sha256_key', None)
    encoded_jpg = annotation_dict.get('encoded_jpg', None)
    image_format = annotation_dict.get('format', None)
    xmins = annotation_dict.get('xmins', None)
    xmaxs = annotation_dict.get('xmaxs', None)
    ymins = annotation_dict.get('ymins', None)
    ymaxs = annotation_dict.get('ymaxs', None)
    masks = annotation_dict.get('masks', None)
    class_names = annotation_dict.get('class_names', None)
    labels = []
    classes_text = []
    for class_name in class_names:
        label = label_map_dict.get(class_name, 'None')
        classes_text.append(class_name.encode('utf8'))
        if label is None:
            raise ValueError('`label_map_dict` is not containing {}.'.format(
                class_name))
        # labels.append(label)
        labels.append(label)
    print('image is {},label is {},'.format(filename, class_names))
    encoded_masks = []
    # print(labels)
    for mask in masks:
        pil_image = PIL.Image.fromarray(mask)
        # pil_image = PIL.Image.fromarray(mask)
        output_io = io.BytesIO()
        pil_image.save(output_io, format='PNG')
        encoded_masks.append(output_io.getvalue())
    feature_dict = {
        'image/height': int64_feature(height),
        'image/width': int64_feature(width),
        'image/filename': bytes_feature(filename.encode('utf8')),
        'image/source_id': bytes_feature(filename.encode('utf8')),
        'image/key/sha256': bytes_feature(sha256_key.encode('utf8')),
        'image/encoded': bytes_feature(encoded_jpg),
        'image/format': bytes_feature('jpeg'.encode('utf8')),#bytes_feature(image_format.encode('utf8')),
        'image/object/bbox/xmin': float_list_feature(xmins),
        'image/object/bbox/xmax': float_list_feature(xmaxs),
        'image/object/bbox/ymin': float_list_feature(ymins),
        'image/object/bbox/ymax': float_list_feature(ymaxs),
        'image/object/class/text':bytes_list_feature(classes_text),
        'image/object/class/label': int64_list_feature(labels),
        'image/object/mask': bytes_list_feature(encoded_masks)
    }
    example = tf.train.Example(features=tf.train.Features(
        feature=feature_dict))
    return example



def get_labelme_json_to_tfrecord(label_map_path,images_dir,annotations_json_dir,output_path):
    label_map = label_map_util.get_label_map_dict(label_map_path)
    writer = tf.python_io.TFRecordWriter(output_path)
    annotations_json_path = os.path.join(annotations_json_dir, '*.json')
    num_annotations_skiped = 0
    for i, annotation_file in enumerate(glob.glob(annotations_json_path)):
        if i % 100 == 0:
            print('On image %d', i)
            
        annotation_dict = _get_annotation_dict(images_dir, annotation_file)
        if annotation_dict is None:
            num_annotations_skiped += 1
            continue
        tf_example = create_tf_example(annotation_dict, label_map)
        writer.write(tf_example.SerializeToString())

    print('Successfully created TFRecord to {}.'.format(output_path))
    
# label_id_map(label_map_path,2)
# annotation_dict = _get_annotation_dict(image_dir, json_dir+"/1ae1f65c098216d1_9-6895.json")
# label_map = label_map_util.get_label_map_dict(label_map_path)
# print(label_map)
# tf_example = create_tf_example(annotation_dict, label_map)

##### 可以运行；

label_map_path = "E:/HongsenQu/2_project/2_projects/images_output/output/new_output/od_data/label.pbtxt"
images_dir = "E:/HongsenQu/2_project/2_projects/images_output/output/new_output/od_data/train/JPEGImages"
annotations_json_dir = 'E:/HongsenQu/2_project/2_projects/images_output/output/new_output/split_dat/train_jsons/'
output_path = "E:/HongsenQu/2_project/2_projects/images_output/output/new_output/od_data/train2.rfrecored"
get_labelme_json_to_tfrecord(label_map_path,images_dir,annotations_json_dir,output_path)

On image %d 0
image is 00410d450fb286f0_9-0906.jpg,label is ['diaodeng', 'diaodeng', 'diaodeng'],
image is 006173050e2dc5bb_9-1587.jpg,label is ['diaodeng', 'diaodeng', 'diaodeng'],
image is 00618c9409d246e7_9-2936.jpg,label is ['pishafa', 'pishafa', 'pishafa', 'pishafa'],
image is 0081707f0fb679de_9-0912.jpg,label is ['diaodeng', 'diaodeng', 'diaodeng', 'diaodeng', 'diaodeng', 'diaodeng'],
image is 00b1e5a501e303fe_9-4120.jpg,label is ['pishafa', 'pishafa'],
image is 00c1865100febee3_9-7703.jpg,label is ['diaodeng', 'diaodeng'],
image is 00e1cd4a0e43f3cb_9-9159.jpg,label is ['pishafa'],
image is 011185b2037f629d_9-5433.jpg,label is ['pishafa', 'pishafa'],
image is 0111d2750ba0fed2_9-5781.jpg,label is ['pishafa', 'pishafa', 'pishafa'],
image is 0111d9ec00fcd9ca_9-7833.jpg,label is ['pishafa'],
image is 01d19c1d0fba54f1_9-9590.jpg,label is ['diaodeng', 'diaodeng', 'diaodeng'],
image is 01f12f100ccb29c5_9-7844.jpg,label is ['pishafa', 'pishafa'],
image is 01f149b004f0ea8f_9-7827.jpg,labe

In [25]:
########################

############# labelme json 转化为voc格式：

import os
import json
import numpy as np
import glob
import shutil
from sklearn.model_selection import train_test_split
from labelme import utils



#0为背景
classname_to_id = {"pishafa": 0 ,
                   "diaodeng" : 1}

def convert(size, box):
    dw = 1. / (size[0])
    dh = 1. / (size[1])
    x = box[0] + box[2] / 2.0
    y = box[1] + box[3] / 2.0
    w = box[2]
    h = box[3]
 
    x = x * dw
    w = w * dw
    y = y * dh
    h = h * dh
    return (x, y, w, h)


json_file = "./00b1e5a501e303fe_9-4120.json"

def labelme_json2yolo_txt(img_dir,json_file):
    data = json.load(open(json_file, 'r'))
    filename = data['imagePath']
    img_width = data["imageWidth"]
    img_heigt = data["imageHeight"]
    annotations = data['shapes']
    
    image_file = os.path.join(img_dir,filename)
    
    if os.path.exists(image_file):
        info = image_file
        for ann in annotations:
            label_name = ann['label']
            points = ann['points']
            Xs = [point[0] for point in points]
            Ys = [point[-1] for point in points]
            # print(points)
            # print(Xs)
            # print(Ys)
            x_min,x_max = min(Xs),max(Xs)
            y_min,y_max = min(Ys),max(Ys)
            box = [x_min, y_min, x_max, y_max]
            label_id = classname_to_id.get(label_name)
            yolo_text = [x_min, y_min, x_max, y_max, label_id]
            # print(yolo_text)
            # print(label_name,label_id)
            
            info+=  ' ' + ','.join([str(x_min), str(y_min), str(x_max), str(y_max), str(label_id)])
        
        print(info)
        return info



# labelme_json2yolo_txt(r'E:/HongsenQu/2_project/2_projects/images_output/output/new_output/images/',json_file)

In [27]:
import glob

train_json_dir = r'E:\HongsenQu\2_project\2_projects\images_output\output\new_output\split_dat\train_jsons/'
img_dir = "E:/HongsenQu/2_project/2_projects/images_output/output/new_output/images/"
save_file = os.path.join("E:/HongsenQu/2_project/2_projects/3_tf_yolo_v3/tensorflow-yolov3-master/data/dataset","voc_train_zc.txt")
jsonList = [os.path.join(train_json_dir,i) for i in os.listdir(train_json_dir) if i.endswith('.json')]

for jfile in jsonList:
    line = labelme_json2yolo_txt(img_dir,jfile)
    print(line,file=open(save_file,"a"))



E:/HongsenQu/2_project/2_projects/images_output/output/new_output/images/00410d450fb286f0_9-0906.jpg 329.1666666666667,132.11009174311926,358.2568807339449,446.02510460251045,1 403.79746835443035,82.91139240506328,437.02531645569616,432.9113924050633,1 503.4810126582278,25.31645569620253,543.0379746835442,417.0886075949367,1
E:/HongsenQu/2_project/2_projects/images_output/output/new_output/images/006173050e2dc5bb_9-1587.jpg 286.2573099415205,111.40350877192982,318.42105263157896,264.03508771929825,1 34.36123348017621,170.92511013215858,109.91189427312776,275.3303964757709,1 431.27753303964755,156.3876651982379,456.1674008810573,279.295154185022,1
E:/HongsenQu/2_project/2_projects/images_output/output/new_output/images/00618c9409d246e7_9-2936.jpg 694.9731543624162,202.4496644295302,899,423.9261744966443,0 380.87919463087246,212.51677852348993,530.5436241610738,259.496644295302,0 325.1744966442953,246.744966442953,445.30872483221475,376.9463087248322,0 113.76510067114094,217.214765100671